# Child in Need Plan (CiNP) LSOA Analysis


# Importing Necessary Libraries

In [ ]:
#import bigquery
from google.cloud import bigquery
from google.cloud import bigquery_storage
 
#other needed libraries
import os
import pandas as pd
import numpy as np
import pandas_gbq
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import mapclassify
from scipy import stats
import contextily as ctx
import libpysal
from pysal.explore import esda
from esda.moran import Moran, Moran_Local
from libpysal.weights import Queen, Rook


import warnings
warnings.filterwarnings('ignore')

plt.style.use('tableau-colorblind10')
color='#702A7D'

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jupyter/.config/gcloud/application_default_credentials.json"

#Instatiate BigQuery Client
client = bigquery.Client()

# Define Query to Get Data

In [ ]:
query = """
WITH LatestAssessment AS (
    SELECT
        a.person_id,
        a.StartDate,
        ROW_NUMBER() OVER (PARTITION BY a.person_id ORDER BY a.StartDate DESC) as rn,
        f.category
    FROM
        yhcr-prd-bradfor-bia-core.CB_2649.tbl_bmbc_Childrens_Social_Services_Assessments AS a
    LEFT JOIN
        yhcr-prd-bradfor-bia-core.CB_2649.cb_FactorLookup AS f
        ON a.factorid = f.factorid
)

SELECT
    cinp.person_id,
    cinp.PCArea_Home AS PCArea,
    cinp.StartDate,
    cinp.EndDate,
    cinp.EthnicOrigin,
    p.birth_datetime AS DateOfBirth,
    lsoa.LSOA,
    la.category as Category
FROM
    yhcr-prd-bradfor-bia-core.CB_2649.cb_bmbc_ChildrensSocialServices_CiNP AS cinp
LEFT JOIN
    yhcr-prd-bradfor-bia-core.CB_2649.person AS p
    ON cinp.person_id = p.person_id
LEFT JOIN
    yhcr-prd-bradfor-bia-core.CB_2649.personLSOA AS lsoa
    ON cinp.person_id = lsoa.person_id
LEFT JOIN
    LatestAssessment la
    ON cinp.person_id = la.person_id
    AND la.rn = 1;
"""